In [12]:
# !pip install -qqq langchain_openai
# !pip install -qqq -U langchain-community
# !pip install -qqq -U langchain_chroma
# !pip install -qqq -U langchain-huggingface
# !pip install -qqq -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [13]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cnamuangtoun/resume-job-description-fit")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [14]:
dataset.column_names

{'train': ['resume_text', 'job_description_text', 'label'],
 'test': ['resume_text', 'job_description_text', 'label']}

In [15]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [18]:
# dataset['train'][0]

In [17]:
print(len(dataset['train']))
print(len(dataset['test']))

6241
1759


In [31]:
db_name = "job_embeddings_db"
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en",encode_kwargs={'normalize_embeddings':True})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
import os

if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function = embeddings).delete_collection()

In [33]:
# Initialize the Chroma vector store
vector_store = Chroma(persist_directory=db_name, embedding_function=embeddings)

# Fetch and store embeddings
for record in dataset['train']:
    resume_text = record['resume_text']
    job_description_text = record['job_description_text']

    # Generate embeddings for resume and job description
    resume_embedding = embeddings.embed_query(resume_text)
    job_description_embedding = embeddings.embed_query(job_description_text)

    # Store embeddings in the vector store with metadata
    vector_store.add_texts(
        texts=[resume_text, job_description_text],
        metadatas=[{'type': 'resume'}, {'type': 'job_description'}]
    )

# Persist the vector store
# vector_store.persist()

In [35]:
embedding_sample = embeddings.embed_query("This is a sample text.")
print(len(embedding_sample))

384


In [ ]:
from langchain_openai import ChatOpenAI

from google.colab import userdata
openai_api = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(api_key=openai_api)

In [41]:
retriever = vector_store.as_retriever()

In [47]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt_search_query = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Based on the above conversation, generate a structured search query to find relevant job postings. "
             "Ensure the query includes key skills, experience level, and location from the user's request.")
])

In [46]:
from langchain.chains import create_history_aware_retriever

retriever_chain = create_history_aware_retriever(llm, retriever, prompt_search_query)

In [48]:
prompt_get_answer = ChatPromptTemplate.from_messages([
    ("system", "You are an AI-powered recruitment assistant. Your goal is to provide job recommendations "
               "based on the user's skills, experience, and location. Use the following job postings "
               "to generate a professional and structured response. Do not return context and system messages.\n\n"
               "{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

In [49]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt_get_answer)

In [50]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [53]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
response = retrieval_chain.invoke({
"chat_history":chat_history,
"input":"I am skilled in Machine learning and Data science with 3 years of experience. Looking for a job in San Francisco Location. Can you provide me some options?"
})
print(response["answer"])

Based on your skills and experience in Machine Learning and Data Science with 3 years of experience, I recommend considering the following job opportunity:

**Role:** Data Engineer (Oracle and DataStage)
**Location:** Remote
**Company:** Cerebral Technologies, Inc (D.B.A CBLSolutions)
**Skills:** Oracle, Datastage, UNIX, PLSQL, SQL
**Preferred Skills:** AWS, Matillion, Snowflake

While the location is remote for this specific job, it aligns well with your expertise and can be a valuable opportunity to utilize your skills in data engineering. If you are open to remote positions and interested in this role, please reach out to Shubhanshu Tripathi at cerebrals solutions for further details.

If you seek on-site positions in San Francisco, please let me know, and I can assist you further in finding suitable opportunities in that specific area.


In [54]:
response2 = retrieval_chain.invoke({
"chat_history":chat_history,
"input":"I am skilled in Java and Python with 8 years of experience. Looking for a job in Mumbai Location. Can you provide me some options?"
})
print(response2["answer"])

Based on your Java and Python skills with 8 years of experience and your preference for a job in Mumbai, I recommend the following opportunity that aligns with your profile:

**Job Role:** Data Engineer (Python or Java/ETL/Hadoop/Spark)
**Location:** Mumbai, India
**Work Mode:** Hybrid (3 days Onsite in a week)
**Duration:** 6+ months (with possible extension or CTH)
**Job Description:** Minimum 7+ years of experience. The role involves migrating existing data pipelines and creating new data pipelines in a cloud environment using Python, Java, Spark, AWS, and SQL.

If you are interested, kindly share your updated resume and contact details with me at your earliest convenience. You can reach out to me at the provided contact information for further discussions.

Thank you!


In [55]:
from langchain_core.messages import HumanMessage, AIMessage

def chat(question, history):
    ai_message = retrieval_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question), ai_message["answer"]])
    return ai_message['answer']

In [57]:
import gradio as gr

gradio_interface = gr.ChatInterface(chat).launch()

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52c82cf4011814ca08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
